# Klasifikasi SMS

Berikut merupakan proses untuk pengelompokan SMS menjadi 3 kategori yaitu normal, penipuan dan penawaran dengan menggunakan algoritma knn.

Ref:
+ Dataset: http://nlp.yuliadi.pro/static/dataset_sms_spam_bhs_indonesia.zip
+ Data stopword: https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.json

## Inisialisasi library 

In [1]:
import string
import re
import json
from collections import Counter

# sklearn - machine learning library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

# numpy - scientific computing library
import numpy as np

# pandas - python data analysis library
import pandas as pd

# sastrawi - stemming library (bahasa indonesia)
# !pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# tqdm - progress bar library
from tqdm import tqdm

## Preprocessing
Membersihkan dokumen dengan cara:
1. Menghilangkan kata yang tidak penting
2. Menghilangkan tanda baca
3. Mengubah kata ke bentuk dasar
4. Menghilangkan angka

In [2]:
# import file yang berisi kata-kata tidak penting
data_stopword = json.load(open('stopwords-id.json','r'))

# menjadikan array stopword menjadi unordered collection (set())
# agar dapat dilakukan operasi matematis seperti union, intersection, symmetric difference
stopword = set(data_stopword)
punctuation = set(string.punctuation)

# method untuk cleaning dokumen
def clean(doc):
    # menghilangkan kata tidak penting
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopword])
    # menghilangkan tanda baca
    punc_free = ''.join(ch for ch in stop_free if ch not in punctuation)
    # menjadikan ke kata dasar
    stemmer = StemmerFactory().create_stemmer()
    normalized = stemmer.stem(punc_free)
    # menghilangkan angka
    processed = re.sub(r"\d+","",normalized)
    # membuat satu dokumen menjadi array berisi tiap kata
    y = processed.split()
    return y

# method untuk cleaning dokumen berupa array
def clean_with_loop(arr):
    hasil = []
    progress = tqdm(arr)
    for item in progress:
        progress.set_description("Membersihkan dokumen")
        cleaned = clean(item)
        cleaned = ' '.join(cleaned)
        hasil.append(cleaned)
    return hasil

In [3]:
# import dataset sms
sms_csv = pd.read_csv('dataset_sms.csv')
print(sms_csv.head())

# mengambil hanya kolom Teks sms dan disimpan di variabel sms (yang siap dibersihkan)
sms = []
for index, row in sms_csv.iterrows():
    sms.append(row["Teks"])
print("Jumlah sms: ", len(sms))

# mengambil hanya kolom label dalam variabel y_train
y_train = []
for index, row in sms_csv.iterrows():
    y_train.append(row["label"])
print("Jumlah label: ", len(y_train))

# membersihkan dokumen sms
sms_bersih = clean_with_loop(sms)

                                                Teks  label
0  [PROMO] Beli paket Flash mulai 1GB di MY TELKO...      2
1  2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...      2
2  2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...      2
3  2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...      2
4  4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...      2
Jumlah sms:  1143
Jumlah label:  1143


Membersihkan dokumen: 100%|████████████████████████████████████████████████████████| 1143/1143 [13:50<00:00,  1.38it/s]


## Membentuk TF-IDF

In [5]:
# pembentukan vektor tf-idf untuk pembobotan kata
vectorizer = TfidfVectorizer(stop_words=data_stopword)
x_train = vectorizer.fit_transform(sms_bersih)
print(x_train)

  (0, 2570)	0.1571563637669152
  (0, 320)	0.16914171127327512
  (0, 2345)	0.1514594440986127
  (0, 935)	0.19437564283726394
  (0, 1020)	0.34162689085952747
  (0, 2101)	0.2441892209184943
  (0, 3176)	0.20094196297969538
  (0, 164)	0.22758469489141753
  (0, 808)	0.20276234267155271
  (0, 905)	0.4622959348047592
  (0, 1734)	0.17081344542976373
  (0, 1848)	0.23936579359516763
  (0, 2134)	0.19592117720194824
  (0, 2044)	0.2634367155575385
  (0, 495)	0.20466742104930438
  (0, 541)	0.14242782069893686
  (0, 3335)	0.2441892209184943
  (0, 2977)	0.2158035961876674
  (1, 2570)	0.20679145834585344
  (1, 1020)	0.2247618908890593
  (1, 2977)	0.2839613955316152
  (1, 2750)	0.19723568879065276
  (1, 2729)	0.26680128970697864
  (1, 3046)	0.2911568296070759
  (1, 2436)	0.24026384399786369
  :	:
  (1138, 2297)	0.3695178514143089
  (1138, 1101)	0.41560616621330004
  (1138, 1578)	0.46169448101229127
  (1138, 3707)	0.4886544168911331
  (1138, 3397)	0.4886544168911331
  (1139, 968)	0.23045532866437865
  (11

Kalau error `IOPub data rate exceeded`
1. Buka **Anaconda Prompt**
2. Jalankan perintah `jupyter notebook --generate-config` untuk generate file config
3. Buka file config yang lokasinya ditampilkan setelah step 2 dieksekusi
4. Cari `c.NotebookApp.iopub_data_rate_limit`
5. Hapus `#` di depan dan ubah nilainya `10000000` (tambah 0 satu kali)

In [6]:
# pengelompokan dokumen dengan knn (k=5)
# penghitungan jarak dengan euclidean distance
# dokumentasi: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
modelknn = KNeighborsClassifier(n_neighbors=5, weights='distance')
modelknn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')

## Pengujian

In [7]:
kalimat_tes = ["INFO RESMI PT-SHOPEE 2018 Selamat No.Anda meraih hadiah ke-2 Rp.175jt dgn kode PIN b8337h9",
               "Besok tahun baruan dimana bro?",
               "Cari tiket pesawat paling gampang dan hemat, ya pakai Airy App. Tinggal tekan, langsung sampai! Ada kode Mudah50 spesial buat kamu"]

# membersihkan dokumen pengujian
kalimat_tes_bersih = clean_with_loop(kalimat_tes)

# definisikan nama label
nama_label = ["normal", "penipuan", "penawaran"]

# loop untuk prediksi kelompok
for teks in kalimat_tes_bersih:
    arr_teks = []
    arr_teks.append(teks)
    vektor = vectorizer.transform(arr_teks)
    prediksi_label_knn = modelknn.predict(vektor)
    print(teks, ":\n" + "- kelompok: " + nama_label[np.int(prediksi_label_knn)]+ "\n")

Membersihkan dokumen: 100%|██████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.81it/s]


info resmi ptshopee selamat noanda raih hadiah ke rpjt dgn kode pin bh :
- kelompok: penipuan

besok baru mana bro :
- kelompok: normal

cari tiket pesawat gampang hemat ya pakai airy app tinggal tekan langsung sampai kode mudah spesial :
- kelompok: penawaran

